In [40]:
import os
import glob
import json
import quickstats
import sys, numpy as np
from quickstats.components import AnalysisBase

In [41]:
# make sure hh combination fw path is defined
hh_comb_fw_path = os.environ.get('hh_combination_fw_path', None)

assert hh_comb_fw_path

gen_command = True

In [42]:
from quickstats.maths.numerics import is_integer, pretty_value
from pprint import pprint
def create_job(string, key, suf=''):
    tmp = string.split(',')
    poi1 = tmp[0].split('=')[0]
    poi2 = tmp[1].split('=')[0]
    poi1_tmp = tmp[0].split('=')[1].split('_')
    poi2_tmp = tmp[1].split('=')[1].split('_')
    poi1_range = [float(p) for p in poi1_tmp[0:2]]
    poi1_step = float(poi1_tmp[2])
    poi2_range = [float(p) for p in poi2_tmp[0:2]]
    poi2_step = float(poi2_tmp[2])

    poi1_nsteps = int((poi1_range[1] - poi1_range[0]) / poi1_step)
    poi2_nsteps = int((poi2_range[1] - poi2_range[0]) / poi2_step)

#     # slice poi1 with full poi2 range
#     if poi1_nsteps < poi2_nsteps and poi2_nsteps > 12:
#         poi1, poi1_range, poi1_step, poi1_nsteps, poi2, poi2_range, poi2_step, poi2_nsteps = poi2, poi2_range, poi2_step, poi2_nsteps, poi1, poi1_range, poi1_step, poi1_nsteps
        
    poi_slice_up = np.arange(poi1_range[0], poi1_range[1], poi1_step).round(decimals=2)
    new_dict = {}
    for i in range(poi1_nsteps):
        key_name = key+f'_job{suf}{i+1}'
        value = f"^{poi1}={poi_slice_up[i]}_{pretty_value(poi_slice_up[i]+poi1_step)}_{poi1_step},{poi2}={pretty_value(poi2_range[0])}_{pretty_value(poi2_range[1])}_{poi2_step}^"
        new_dict[key_name] = value
    return new_dict


## Step 1: Configure task

In [43]:
# specify the input timestamp to use
timestamp = "20220520"
withbr = 'with' # 'without'

paths = {
    'input'              : os.path.join(hh_comb_fw_path, "FullRun2Workspaces", "original", "HHH2022", timestamp),
    'task_options'       : os.path.join(hh_comb_fw_path, "configs", "task_options"       , "HHH2022",
                                        f"nonres_kl_kt_likelihood_{withbr}_BR_decorrelation.yaml"),
    'correlation_schemes': os.path.join(hh_comb_fw_path, "configs", "correlation_schemes", "HHH2022", 
                                        "nonres_kl_v12.json"),
    #'output'             : os.path.join(os.getcwd(), f"outputs_HHH2022_{timestamp}")
    'output'             : os.path.join(os.getcwd(), f"outputs_HHH2022_{timestamp}_{withbr}_BR_decorrelation")
}

config = {
    'resonant_type': 'nonres',
    'channels'     : ['bbbb', 'bbtautau', 'bbyy'],
    'file_expr'    : '<mass[F]>_kl',
    'blind'        : False,
    'cache'        : True,
    'experimental' : True,
    'parallel'     : -1
}

options = {
    "input_dir"    : paths['input'],
    "resonant_type": config['resonant_type'],
    "channels"     : ",".join(config['channels']),
    "outdir"       : paths['output'],
    "file_expr"    : f"\"{config['file_expr']}\"",
    "config"       : paths['task_options'],
    "parallel"     : config['parallel'],
    "skip-limit"   : ""
}

## Step 2: Prepare modified channel workspaces

In [44]:
if config["blind"]:
    options["blind"] = ""
else:
    options["unblind"] : ""
if config["experimental"]:
    options["experimental"] = ""
else:
    options["official"] : ""
if config['cache']:
    options['cache'] = ""
else:
    options['no-cache'] = ""
command_str = "HHComb process_channels " + " ".join([f"--{key} {value}" for key, value in options.items()])
print(command_str)

HHComb process_channels --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/FullRun2Workspaces/original/HHH2022/20220520 --resonant_type nonres --channels bbbb,bbtautau,bbyy --outdir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation --file_expr "<mass[F]>_kl" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_kl_kt_likelihood_with_BR_decorrelation.yaml --parallel -1 --skip-limit  --experimental  --cache 


In [45]:
if not gen_command:
    os.system(command_str)

## Step 3: Prepare combined workspace

In [46]:
def return_combine_command(paths, config):
    options = {
        "input_dir"    : paths['output'],
        "resonant_type": config['resonant_type'],
        "channels"     : ",".join(config['channels']),
        "file_expr"    : f"\"{config['file_expr']}\"",
        "config"       : paths['task_options'],
        "scheme"       : paths['correlation_schemes'],
        "parallel"     : config['parallel'],
        "skip-limit"   : ""
    }
    if config["blind"]:
        options["blind"] = ""
    else:
        options["unblind"] : ""
    if config["experimental"]:
        options["experimental"] = ""
    else:
        options["official"] : ""
    if config['cache']:
        options['cache'] = ""
    else:
        options['no-cache'] = ""
    command_str = "HHComb combine_ws " + " ".join([f"--{key} {value}" for key, value in options.items()])
    print(command_str)
    return command_str

command_str = return_combine_command(paths, config)

HHComb combine_ws --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation --resonant_type nonres --channels bbbb,bbtautau,bbyy --file_expr "<mass[F]>_kl" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_kl_kt_likelihood_with_BR_decorrelation.yaml --scheme /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/correlation_schemes/HHH2022/nonres_kl_v12.json --parallel -1 --skip-limit  --experimental  --cache 


In [47]:
if not gen_command:
    os.system(command_str)

**Checkout the rescaled workspace paths**

In [48]:
rescaled_ws_paths = {}
for channel in config['channels']:
    ws_path = os.path.join(paths['output'], 'rescaled', config['resonant_type'], channel, "0_kl.root")
    rescaled_ws_paths[channel]  = ws_path
# rescaled workspace path for combined workspacee
ws_path = glob.glob(os.path.join(paths['output'], 'combined', config['resonant_type'], "*", "0_kl.root"))[0]
rescaled_ws_paths["combined"] = ws_path
rescaled_ws_paths

{'bbbb': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbbb/0_kl.root',
 'bbtautau': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbtautau/0_kl.root',
 'bbyy': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbyy/0_kl.root',
 'combined': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_kl.root'}

## Step 4: Prepare Asimov workspaces

**Set up the asimov workspace paths**

In [49]:
asimov_ws_paths = {}
for channel in rescaled_ws_paths:
    rescaled_ws_path = rescaled_ws_paths[channel]
    asimov_ws_path   = os.path.join(os.path.dirname(rescaled_ws_path), "0_asimov.root")
    asimov_ws_paths[channel] = asimov_ws_path
asimov_ws_paths

{'bbbb': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbbb/0_asimov.root',
 'bbtautau': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbtautau/0_asimov.root',
 'bbyy': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbyy/0_asimov.root',
 'combined': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_asimov.root'}

**Generate CLI commands**

In [50]:
options = {
    'poi'         : 'xsec_br',
    'data'        : 'combData',
    'asimov_types': '2'
}
command_str_map = {}
for channel in rescaled_ws_paths:
    input_file  = rescaled_ws_paths[channel]
    output_file = asimov_ws_paths[channel]
    channel_options = {"input_file": input_file, "output_file": output_file, **options}
    channel_command_str = "quickstats generate_standard_asimov " + \
                          " ".join([f"--{key} {value}" for key, value in channel_options.items()])
    command_str_map[channel] = channel_command_str
    print(channel_command_str, '&')
    print()

quickstats generate_standard_asimov --input_file /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbbb/0_kl.root --output_file /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbbb/0_asimov.root --poi xsec_br --data combData --asimov_types 2 &

quickstats generate_standard_asimov --input_file /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbtautau/0_kl.root --output_file /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbtautau/0_asimov.root --poi xsec_br --data combData --asimov_types 2 &

quickstats generate_standard_asimov --input_file /afs/cern.ch/work/z/zhangr/HHcomb/hh_combinat

**Generate asimov data**

In [51]:
if not gen_command:
    for channel, command_str in command_str_map.items():
        print(f"INFO: Generating asimov data for the channel \"{channel}\"")
        os.system(command_str)

## Step 5: Prepare post-fit snapshots

**Set up the fitted workspace paths**

In [52]:
fitted_ws_paths = {}
for channel in asimov_ws_paths:
    asimov_ws_path = asimov_ws_paths[channel]
    fitted_ws_path = os.path.join(os.path.dirname(asimov_ws_path), "0_fitted.root")
    fitted_ws_paths[channel] = fitted_ws_path
fitted_ws_paths

{'bbbb': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbbb/0_fitted.root',
 'bbtautau': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbtautau/0_fitted.root',
 'bbyy': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbyy/0_fitted.root',
 'combined': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_fitted.root'}

In [53]:
pois_to_scan = ["klambda", "kt", "kV", "k2V", "kF", "kH", "ktau", "klambda,kt", "kV,k2V"]
minimizer_options = {
    "retry": 2,
    "eps": 1
}
# observed and expected datasets
datasets = {
    "observed": "combData",
    "expected": "asimovData_muhat_NP_Profile"
}

In [54]:
# parallel channels
import multiprocessing

def run_best_fit(channel):
    print(f"==> Channel: {channel}")
    asimov_ws_path = asimov_ws_paths[channel]
    fitted_ws_path = fitted_ws_paths[channel]
    kwargs = {
        "filename" : asimov_ws_path,
        # only need to generate snapshot for observed data
        "data_name": datasets["observed"],
        "poi_name": [],
        "config": {**minimizer_options}
    }
    analysis = AnalysisBase(**kwargs)
    # fix all pois at the beginning
    analysis.setup_parameters(fix_param="<pois>")
    analysis.save_snapshot(analysis.kTempSnapshotName)
    for poi_expr in pois_to_scan:
        print(f"==> POIs: {poi_expr}")
        analysis.setup_parameters(profile_param=poi_expr)
        analysis.nll_fit(mode=3)
        pois = poi_expr.split(",")
        snapshot_name = f"obs_bestfit_{'_'.join(pois)}"
        # save best-fit snapshot
        analysis.save_snapshot(snapshot_name)
        analysis.load_snapshot(analysis.kTempSnapshotName)
    # restore initial snapshot
    analysis.load_snapshot(analysis.kInitialSnapshotName)
    analysis.save(fitted_ws_path)

def driver_func():
    PROCESSES = len(fitted_ws_paths)
    with multiprocessing.Pool(PROCESSES) as pool:
        channels = list(fitted_ws_paths.keys())
        channels = ['bbtautau', 'combined']
        results = [pool.apply_async(run_best_fit, (c, )) for c in channels]

        for r in results:
            print('\t', r.get())

if not gen_command:
    driver_func()



- By now, you should have the workspace `0_fitted.root` for each channel
- Inside `0_fitted.root`, you have the observed dataset `combData` and the asimov dataset `asimovData_muhat_NP_Profile`
- For each dataset, there are the corresponding muhat snapshots designed for each likelihood scan, e.g.
  - obs_bestfit_klambda, obs_bestfit_kt, obs_bestfit_kV, obs_bestfit_k2V, obs_bestfit_klambda_kt, obs_bestfit_kV_k2V
  - exp_bestfit_klambda, exp_bestfit_kt, exp_bestfit_kV, exp_bestfit_k2V, exp_bestfit_klambda_kt, exp_bestfit_kV_k2V

## Step 6: Run likelihood scans

**Only commands are shown here since the jobs are computationally intensive**

In [67]:
from quickstats.parsers import ParamParser

# use '_job' + suffix to split scan range
param_expr_maps = {
#     'klambda': "^klambda=-6_12_0.2^",
#     "klambda_kt_profile": "^klambda=-16_20_0.4,kt^",
    "klambda_kt_2D_final": "^klambda=-15_20_0.2,kt=-2_3_0.2^",
#     "kt": "^kt=-2_6_0.1^",
#     "kF": "^kF=-8_15_0.1^",
#     "kV": "^kV=-2.5_2.5_0.1^",
#     "k2V": "^k2V=-3_5_0.1^",
#     "kV_k2V_2D_final": "^kV=-3_3_0.2,k2V=-4_10_0.2^",
}

# param_expr_maps.update(create_job('klambda=-15_20_0.2,kt=-2_3_0.2', 'klambda_kt_2D'))
# param_expr_maps.update(create_job('kV=-3_3_0.2,k2V=-4_10_0.2', 'kV_k2V_2D'))
pprint(param_expr_maps)

{'klambda_kt_2D_final': '^klambda=-15_20_0.2,kt=-2_3_0.2^'}


## finer jobs

In [68]:
# param_expr_maps = {}
# param_expr_maps.update(create_job('klambda=-15_20_0.2,kt=-2_-1_0.2', 'klambda_kt_2D', suf='a'))
# param_expr_maps.update(create_job('klambda=-15_20_0.2,kt=-1_0_0.2', 'klambda_kt_2D', suf='b'))
# param_expr_maps.update(create_job('klambda=-15_20_0.2,kt=0_1_0.2', 'klambda_kt_2D', suf='c'))
# param_expr_maps.update(create_job('klambda=-15_20_0.2,kt=1_2_0.2', 'klambda_kt_2D', suf='d'))
# param_expr_maps.update(create_job('klambda=-15_20_0.2,kt=2_3_0.2', 'klambda_kt_2D', suf='e'))
# param_expr_maps.update(create_job('kV=-3_-2_0.2,k2V=-4_10_0.2', 'kV_k2V_2D', suf='a'))
# param_expr_maps.update(create_job('kV=-2_-1_0.2,k2V=-4_10_0.2', 'kV_k2V_2D', suf='b'))
# param_expr_maps.update(create_job('kV=-1_0_0.2,k2V=-4_10_0.2', 'kV_k2V_2D', suf='c'))
# param_expr_maps.update(create_job('kV=0_1_0.2,k2V=-4_10_0.2', 'kV_k2V_2D', suf='d'))
# param_expr_maps.update(create_job('kV=1_2_0.2,k2V=-4_10_0.2', 'kV_k2V_2D', suf='e'))
# param_expr_maps.update(create_job('kV=2_3_0.2,k2V=-4_10_0.2', 'kV_k2V_2D', suf='f'))
# print(len(param_expr_maps))
# pprint(param_expr_maps)

In [69]:
condor = False

likelihood_scan_cmds = {}
for channel in fitted_ws_paths:
    likelihood_scan_cmds[channel] = {}
    fitted_ws_path = fitted_ws_paths[channel]
    for dataset_type, dataset_name in datasets.items():
        likelihood_scan_cmds[channel][dataset_type] = {}
        for key, expr in param_expr_maps.items():
            if condor and 'final' in key:
                continue
            if not condor and 'job' in key:
                continue
            if 'final' in key:
                outdir = os.path.join(paths['output'], 'likelihood', dataset_type, channel, key.replace('_final', ''))
            else:
                outdir = os.path.join(paths['output'], 'likelihood', dataset_type, channel, (key+'1')[:key.find('_job')])
            pois = ParamParser._get_param_str_attributes(expr)
            pois = [p.replace('^', '') for p in pois]
            if dataset_type == "observed":
                snapshot_name = f"obs_bestfit_{'_'.join(pois)}"
            else:
                snapshot_name = dataset_name
            options = {
                "input_file" : fitted_ws_path,
                "data": dataset_name,
                "param_expr": expr,
                'outdir': outdir,
                'snapshot': snapshot_name,
                **minimizer_options
            }
            cmd_str = "quickstats likelihood_scan " + \
                      " ".join([f"--{key} {value}" for key, value in options.items()])
            likelihood_scan_cmds[channel][dataset_type][key] = cmd_str
            
            
if condor:
    print('save to', f'job{withbr}.txt')
    original_stdout = sys.stdout
    sys.stdout = open(f'job{withbr}.txt', 'w')
for channel in likelihood_scan_cmds:
    if not condor:
        print("#########################################################################################")
    print(f"# Channel: {channel}")
    for dataset_type in likelihood_scan_cmds[channel]:
        print(f"# Dataset: {dataset_type}")
        for key in likelihood_scan_cmds[channel][dataset_type]:
            if not condor:
                print(f"# Scan parameters: {key}")
            cmd_str = likelihood_scan_cmds[channel][dataset_type][key]
            if not condor:
                print(cmd_str.replace('^', '"'))
            else:
                print("Arguments =", cmd_str.replace(' ', '____'))
                print("Queue 1")
        print()
    if not condor:
        print("#########################################################################################")

if condor: 
    sys.stdout = original_stdout

#########################################################################################
# Channel: bbbb
# Dataset: observed
# Scan parameters: klambda_kt_2D_final
quickstats likelihood_scan --input_file /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbbb/0_fitted.root --data combData --param_expr "klambda=-15_20_0.2,kt=-2_3_0.2" --outdir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/likelihood/observed/bbbb/klambda_kt_2D --snapshot obs_bestfit_klambda_kt --retry 2 --eps 1

# Dataset: expected
# Scan parameters: klambda_kt_2D_final
quickstats likelihood_scan --input_file /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbbb/0_fitted.root --data asimovData_muhat_NP_Profile --param_expr "klambda=-

## Best fit

In [34]:
from quickstats.components import AnalysisBase


In [35]:
if "hh_combination_fw_path" not in os.environ:
    os.environ['hh_combination_fw_path'] = os.path.abspath("../../")
basedir = os.environ['hh_combination_fw_path']+'/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/'

ws_paths = {
    "bbbb": os.path.join(basedir, "rescaled/nonres/bbbb/0_fitted.root"),
    "bbtautau": os.path.join(basedir, "rescaled/nonres/bbtautau/0_fitted.root"),
    "bbyy": os.path.join(basedir, "rescaled/nonres/bbyy/0_fitted.root"),
    "combined": os.path.join(basedir, "combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_fitted.root")
}

In [36]:
poi_scenarios = ["klambda", ["klambda", "kt"], "k2V", ["kV", "k2V"], "kF", "kt", "kV"]
channels = ["bbbb", "bbtautau", "bbyy", "combined"]
datasets = ["observed", "expected"]

In [37]:
bestfit_results = {}
for poi_scenario in poi_scenarios:
    
    if isinstance(poi_scenario, list):
        poi_str = "_".join(poi_scenario)
    else:
        poi_str = poi_scenario
    bestfit_results[poi_str] = {}
    for channel in channels:
        bestfit_results[poi_str][channel] = {}
        for dataset in datasets:
            print(f"Scenario: {poi_scenario}, Channel: {channel}, Dataset: {dataset}")
            if dataset == "observed":
                data_name = "combData"
                if isinstance(poi_scenario, str):
                    snapshot_name = f"obs_bestfit_{poi_scenario}"
                else:
                    snapshot_name = f"obs_bestfit_" + "_".join(poi_scenario)
            else:
                data_name = "asimovData_muhat_NP_Profile"
                snapshot_name = "asimovData_muhat_NP_Profile"
            outdir = os.path.join(basedir, "bestfit", dataset, channel, poi_str)
            if not os.path.exists(outdir):
                os.makedirs(outdir)
            outpath = os.path.join(outdir, "bestfit.json")
            if os.path.exists(outpath):
                continue
            analysis = AnalysisBase(ws_paths[channel], data_name=data_name, poi_name=poi_scenario,
                        config={"snapshot_name": snapshot_name})
            result = analysis.nll_fit(mode=1, do_minos=True)
            with open(outpath, "w") as out:
                json.dump(result, out, indent=2)

Scenario: klambda, Channel: bbbb, Dataset: observed
Scenario: klambda, Channel: bbbb, Dataset: expected
Scenario: klambda, Channel: bbtautau, Dataset: observed
Scenario: klambda, Channel: bbtautau, Dataset: expected
Scenario: klambda, Channel: bbyy, Dataset: observed
Scenario: klambda, Channel: bbyy, Dataset: expected
Scenario: klambda, Channel: combined, Dataset: observed
Scenario: klambda, Channel: combined, Dataset: expected
Scenario: ['klambda', 'kt'], Channel: bbbb, Dataset: observed
Scenario: ['klambda', 'kt'], Channel: bbbb, Dataset: expected
Scenario: ['klambda', 'kt'], Channel: bbtautau, Dataset: observed
Scenario: ['klambda', 'kt'], Channel: bbtautau, Dataset: expected
Scenario: ['klambda', 'kt'], Channel: bbyy, Dataset: observed
Scenario: ['klambda', 'kt'], Channel: bbyy, Dataset: expected
Scenario: ['klambda', 'kt'], Channel: combined, Dataset: observed
Scenario: ['klambda', 'kt'], Channel: combined, Dataset: expected
Scenario: k2V, Channel: bbbb, Dataset: observed
Scenario

INFO: NLL evaluation completed with
    best fit : kF=1.00000
  uncond NLL = 290930.13013
        time = (uncond_fit) 37.318, (cond_fit) 0.000
Scenario: kF, Channel: bbyy, Dataset: observed
INFO: Opening file "/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbyy/0_fitted.root"
INFO: Loaded workspace "combWS"
INFO: Loaded model config "ModelConfig"
INFO: Loaded model pdf "CombinedPdf" from model config
INFO: Loaded dataset "combData" from workspace
INFO: Loaded nuisance parameters from model config
INFO: Loaded global observables from model config
INFO: Loaded parameters of interest from model config
INFO: Loaded observables from model config
INFO: Loaded snapshot "obs_bestfit_kF"
INFO: POI set to "kF"
INFO: Created ExtendedMinimizer("Minimizer") instance
INFO: NLL evaluation completed with
    best fit : kF=4.35799
  uncond NLL = 522.20833
        time = (uncond_fit) 6.520, (cond_fit)

INFO: POI set to "kt"
INFO: Created ExtendedMinimizer("Minimizer") instance
INFO: NLL evaluation completed with
    best fit : kt=-0.00568
  uncond NLL = 52486.38703
        time = (uncond_fit) 275.753, (cond_fit) 0.000
Scenario: kt, Channel: bbbb, Dataset: expected
INFO: Opening file "/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/rescaled/nonres/bbbb/0_fitted.root"
INFO: Loaded workspace "combWS"
INFO: Loaded model config "ModelConfig"
INFO: Activated binned likelihood attribute for ggf_16_dEta_1_Xhh_1_model
INFO: Activated binned likelihood attribute for ggf_16_dEta_1_Xhh_2_model
INFO: Activated binned likelihood attribute for ggf_16_dEta_2_Xhh_1_model
INFO: Activated binned likelihood attribute for ggf_16_dEta_2_Xhh_2_model
INFO: Activated binned likelihood attribute for ggf_16_dEta_3_Xhh_1_model
INFO: Activated binned likelihood attribute for ggf_16_dEta_3_Xhh_2_model
INFO: Activated binned lik

INFO: Loaded snapshot "obs_bestfit_kt"
INFO: POI set to "kt"
INFO: Created ExtendedMinimizer("Minimizer") instance
INFO: NLL evaluation completed with
    best fit : kt=0.13142
  uncond NLL = 756931.64550
        time = (uncond_fit) 340.067, (cond_fit) 0.000
Scenario: kt, Channel: combined, Dataset: expected
INFO: Opening file "/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_with_BR_decorrelation/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_fitted.root"
INFO: Loaded workspace "combWS"
INFO: Loaded model config "ModelConfig"
INFO: Activated binned likelihood attribute for ggf_16_dEta_1_Xhh_1_model_bbbb
INFO: Activated binned likelihood attribute for ggf_16_dEta_2_Xhh_1_model_bbbb
INFO: Activated binned likelihood attribute for ggf_16_dEta_3_Xhh_1_model_bbbb
INFO: Activated binned likelihood attribute for ggf_16_dEta_1_Xhh_2_model_bbbb
INFO: Activated binned likelihood attribute for ggf_16_dEta_2_Xhh_2_model_bbbb
INFO: A

Error in <ROOT::Math::Fitter::CalculateMinosErrors>: Minos error calculation failed for all the selected parameters
Error in <ROOT::Math::Fitter::CalculateMinosErrors>: Minos error calculation failed for all the selected parameters
Error in <ROOT::Math::Fitter::CalculateMinosErrors>: Minos error calculation failed for all the selected parameters
